In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Step 1: Install And Imprort Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import tensorflow as tf
from tensorflow import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,MaxPooling2D,BatchNormalization,Dropout,GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.applications import VGG19
from keras.utils import image_dataset_from_directory
from keras.applications.vgg19 import preprocess_input

# Step 2: Read the Dataset

In [ ]:
# Directories for training and test data
training_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Training'  # loading the directories
test_dir = '/kaggle/input/fruits/fruits-360_dataset_100x100/fruits-360/Test'

In [ ]:
# Number of training and test images
image_files = glob(training_dir + '/*/*.jp*g')
test_image_files = glob(test_dir + '/*/*.jp*g')

# Number of classes
folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))

# Read Data and Data Augumentation

In [ ]:
# Image Augmentation
from keras.applications import VGG19
from keras.utils import image_dataset_from_directory
from keras.applications.vgg19 import preprocess_input

In [ ]:
# Load the data and create training/testing datasets
IMAGE_SIZE = [224, 224]  # Standard size for VGG19
BATCH_SIZE = 200

In [ ]:
test_dir

In [ ]:
training_dir

In [ ]:
training_generator = image_dataset_from_directory(
    training_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)
training_generator

In [ ]:
test_generator = image_dataset_from_directory(
    test_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical"
)
test_generator

# Step 3: Data processing, Data Augumentation

# Step 4: Build the VGG19 Model

In [ ]:
from keras.models import Model

vgg19 = VGG19(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)   # Load the pre-trained VGG19 model

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg19.layers:                                        # Freeze the layers of the pre-trained VGG19 model
    layer.trainable = False

# Add custom layers on top of the VGG19 base
x = Flatten()(vgg19.output)
x = Dense(4096, activation='relu')(x)           # here Fully connected layer
x = Dropout(0.5)(x)                             # here Dropout for regularization
x = Dense(1096, activation='relu')(x)           # here again Fully connected layer
x = Dropout(0.5)(x)                             # here again Dropout for regularization
x = Dense(num_classes, activation='softmax')(x)  # Output layer

model = Model(inputs=vgg19.input, outputs=x)

# here we Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0001),
    metrics=['accuracy']
)
model.summary()

# Step5: Model Training & Testing

In [ ]:
# Train the model
history = model.fit(
    training_generator,
    steps_per_epoch=10000,                  # Adjust based on training dataset size     # this should be equal to total number of images in training set. But to speed up the execution, I am only using 10000 images. Change this for better results.
    epochs=1,                               # Set higher epochs for better results
    validation_data=test_generator,
    validation_steps=3000                   # Adjust based on validation dataset size
)                                           # this should be equal to total number of images in validation set.

# Step 6 : Display Result

In [ ]:
print ('Training Accuracy = ' + str(history.history['accuracy']))
print ('Validation Accuracy = ' + str(history.history['val_accuracy']))

In [ ]:
# Plot training and validation metrics
plt.figure(figsize=(12, 4))

# Accuracy plot
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Loss plot
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()


In [ ]:
# Save the trained model
model.save('vgg19_fruit_classifier.h5')
print("Model saved as vgg19_fruit_classifier.h5")